In [ ]:
# Lung Cancer Detection - Inference with Best Model

import torch
import torch.nn as nn
from torchvision import models, datasets, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import os

# 1️⃣ Device 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("🚀 Using device:", device)

# 2️⃣ Transform 정의
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# 3️⃣ Balanced Test Dataset 경로 설정 (상대경로)
balanced_test_dir = "./balanced_test"

# 4️⃣ Dataset / DataLoader
test_dataset = datasets.ImageFolder(balanced_test_dir, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

print(f"📊 테스트 데이터셋: {len(test_dataset)}장 (클래스: {test_dataset.classes})")

# 5️⃣ ResNet50 모델 로드
model = models.resnet50(weights=None)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 1)  # Binary Classification

model.load_state_dict(torch.load("./resnet50_lung_cancer.pth", map_location=device))
model = model.to(device)
model.eval()

# 6️⃣ 예측 수행
all_preds = []
all_labels = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        outputs = model(inputs)
        preds = torch.sigmoid(outputs).cpu().numpy() > 0.5

        all_preds.extend(preds.astype(int).flatten().tolist())
        all_labels.extend(labels.numpy().tolist())

# 7️⃣ 결과 출력
acc = accuracy_score(all_labels, all_preds)
cm = confusion_matrix(all_labels, all_preds)

print(f"\n✅ Test Accuracy: {acc:.4f}")
print("Confusion Matrix:")
print(cm)

# Classification Report
print("\nClassification Report:")
print(classification_report(all_labels, all_preds, target_names=test_dataset.classes))
